<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/main/ex12noteA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex12noteA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2022)


----
## 準備
----

Google Colab の Notebook では， Python というプログラミング言語のコードを動かして計算したりグラフを描いたりできます．
Python は，機械学習・人工知能やデータサイエンスの分野ではメジャーなプログラミング言語ですが，それを学ぶことはこの授業の守備範囲ではありません．以下の所々に現れるプログラムっぽい記述の内容は，理解できなくて構いません．

以下，コードセルを上から順に実行してながら読んでいってね．

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# SciPy の階層型クラスタリングパッケージ
import scipy.cluster.hierarchy as hierarchy

----
## クラスタリング(2) 階層型クラスタリング
----




---
### 階層型クラスタリングと非階層型クラスタリング





クラスタリングの手法は，**階層型クラスタリング** と **非階層型クラスタリング** に大別できます．

**階層型クラスタリング** (hierarchical clustering): 「クラスタAとクラスタBをあわせたものがクラスタPで，クラスタPとクラスタQをあわせたものがクラスタR」というように，階層的になったクラスタを作るクラスタリング手法．
さらに以下の二つに分けられる：
- **凝集型クラスタリング** (agglomerative clustering): 個々のデータをそれぞれ一つのクラスタとした状態から始めて，だんだんクラスタをくっつけていく 
- **分割型（分枝型）クラスタリング** (divisive clustering) : 全てのデータを一つのクラスタとした状態から始めて，それを分割していく 

**非階層型クラスタリング** (non-hierarchical clustering): クラスタ同士に上記のような階層構造をつくらないクラスタリング手法．「クラスタリング(1)」で紹介した**$K$-平均法** が代表例．様々な手法があり，性質やアルゴリズムによって細分類できるが，この授業では省略する．





凝集型クラスタリングの一例を以下の図に示します．詳しい説明はまた後でします．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/hierarchy-anim.gif">

---
### 凝集型クラスタリング

凝集型クラスタリングについて説明します．この授業では，分割型クラスタリングについては説明を省略します．

#### 凝集型クラスタリングの手順とデンドログラム

凝集型クラスタリングの手順は，大まかには次のようになります．

(0) 個々の学習データをそれぞれクラスタとする．

(1) 現在のクラスタたちの間の距離を求め，最も距離の小さかった二つのクラスタを合併して一つのクラスタにする．

(2) クラスタが一つだけになっていたら終了．さもなくば(1) へ戻る．

凝集型クラスタリングでは，「クラスタとクラスタの間の距離」をどのように測るかが重要なポイントで，様々な方法が考えられています（あとで説明します）．

階層型クラスタリングでは，クラスタリングを行って得られるクラスタの階層構造を調べて，データの特徴を分析することになります．クラスタの階層構造は，**デンドログラム**（**樹形図**，dendrogram）という図に表すことがよくあります．

以下の図の右は，左のデータに凝集型クラスタリングを適用して得られたクラスタの構造を表すデンドログラムです（↑に載せてるものの再掲）．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/hierarchy-anim.gif">

このデンドログラムの縦軸は，クラスタ間の距離を表しています．1番と2番のデータに注目すると，下の方では別々の縦線が描かれていたものが，少し上にたどると一つに合わさっています．これは，この地点でこの二つのデータ（のそれぞれのみから成るクラスタ）が合わさって一つのクラスタになることを意味しています．このときの縦軸の値は，両者の間の距離を表しています．

そのままデンドログラムを上に登っていくと，次は「1番と2番の合わさったクラスタ」と3番のデータが一つのクラスタになることも分かりますね．以下，同様にして，だんだんとクラスタが合併していく様子を見ることができます．

階層型クラスタリングでは，このような樹形図を見て，データを最終的にいくつのクラスタに分けるかを判断することができます．この例では，距離の基準（しきい値）を $3$ とすれば，「0-4番を含むクラスタ」と「5-9番を含むクラスタ」の2つに分けられますし，$2$ とすれば，「0-3番を含むクラスタ」，「4番のみのクラスタ」，「5,6,7,9番を含むクラスタ」，「8番のみのクラスタ」の4つに分けられます．
このように，結果を見てクラスタの分け方を選べるのが階層型クラスタリングの特徴です．

#### クラスタ間の距離の測り方

先にも書いたように，階層型クラスタリングにおいては，「クラスタとクラスタの間の距離」をどのように測るかが重要なポイントとなります．
データの性質に応じて様々なものが考えられます（注）が，
量的な特徴量のみから成るデータの場合，データ点間の距離を定めて，それに基づいてクラスタ間の距離を定める，というのが一般的です．

<span style="font-size: 75%">
※注: 詳しくは説明しませんが，質的な特徴量を，one-hot符号化したりせずそのまま扱えるような方法もあります．
</span>



以下，「クラスタとクラスタの間の距離」の測り方について，いくつかの方法を紹介しますが，個別の方法を覚えることは重要ではありません（実際に使うことになったときにあらためて調べればよいでしょう）．

量的なデータのデータ点間の距離としては，おなじみのユークリッド距離などがよく用いられます．二つのデータを
$$
\mathbf{x} = (x_1, x_2, \ldots, x_D),\quad \mathbf{y} = (y_1, y_2, \ldots, y_D)
$$
として，これらの間の距離を $d(\mathbf{x}, \mathbf{y})$ と表すとき，ユークリッド距離ならば
$$
d(\mathbf{x}, \mathbf{y}) = \sqrt{ \sum_{d=1}^{D}(x_d - y_d)^2 }
$$
ですね．他にも，次式で表される「マンハッタン距離(Manhattan distance)」（注）なども用いられます．
$$
d(\mathbf{x}, \mathbf{y}) = \sum_{d=1}^{D}|x_d - y_d|
$$

<span style="font-size: 75%">
※注: 余談ですが，ユークリッド距離やマンハッタン距離を一般化したものとして，「ミンコフスキー距離」というものがあります．定義は
$$
d(\mathbf{x}, \mathbf{y}) = \left( \sum_{d=1}^{D}|x_d - y_d|^{p} \right)^{\frac{1}{p}}
$$
です．$p = 2$ ならユークリッド距離，$p=1$ ならマンハッタン距離です．総称して，「$L_p$距離」という言い方をすることもあります．ユークリッド距離は $L_2$ 距離，マンハッタン距離は $L_1$ 距離．
</span>


クラスタとクラスタの間の距離の測り方も，様々なものがあります．
代表的なものをいくつか挙げておきます．

<dl>
<dt><b>単リンク法</b>（single linkage method）</dt>
<dd>クラスタ$C_1$とクラスタ$C_2$ の間の距離 $d(C_1, C_2)$を次式で測る方法：
$$
d(C_1, C_2) = \min_{\mathbf{x}_1\in C_1, \mathbf{x}_2\in C_2} d(\mathbf{x}_1, \mathbf{x}_2)
$$
つまり，2つのクラスタからひとつずつ任意にデータを選んで距離を測ったときに，その最小値をクラスタ間の距離とする．
</dd>
<dt><b>完全リンク法</b>（complete linkage method）</dt>
<dd>クラスタ$C_1$とクラスタ$C_2$ の間の距離 $d(C_1, C_2)$を次式で測る方法：
$$
d(C_1, C_2) = \max_{\mathbf{x}_1\in C_1, \mathbf{x}_2\in C_2} d(\mathbf{x}_1, \mathbf{x}_2)
$$
単リンク法と逆で，2つのクラスタからひとつずつ任意にデータを選んで距離を測ったときに，その最大値をクラスタ間の距離とする．
</dd>
<dt><b>ウォード法</b>（Ward's method）</dt>
<dd>階層型クラスタリングでよく用いられる方法．ここでは，どのような距離の測り方をするのかについての説明は省略します．</dd>
</dl>

よく用いられるものはこれ以外にもありますが，説明は省略します．
興味があれば，以下のリンクをたどってみてね．
- Python の科学技術計算ライブラリ SciPy https://scipy.org/
    - の中の階層型クラスタリングパッケージ [scipy.cluster.hierarchy](https://docs.scipy.org/doc/scipy/reference/cluster.hierarchy.html)
        - の中のクラスタリングの関数 [scipy.cluster.hierarchy.linkage](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html#scipy.cluster.hierarchy.linkage)． 様々なクラスタ間距離の測り方が使えるようになっていて，その説明もあります．
- Python の機械学習ライブラリ scikit-learn https://scikit-learn.org/
    - の階層型クラスタリングに関するドキュメント https://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering
    - 凝集型クラスタリングのクラス [sklearn.cluster.AgglomerativeClustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html)

#### 実験: 別の2次元データでもやってみよう

「クラスタリング(1)」で $K$-平均法の実験に使ったのと同じデータに凝集型クラスタリングのアルゴリズムを適用してみましょう．

In [ ]:
# 実験用データの入手
df = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/data4kmeans.csv')
dat1 = df[['x1', 'x2']].to_numpy()
dat2 = df[['y1', 'y2']].to_numpy()
X2 = dat1

次のセルを実行すると，クラスタリングを行った結果を表示します．

In [ ]:
#@title 2次元データの階層型クラスタリング
#@markdown `method` でクラスタ間距離の計算法を指定．
#@markdown  - `single` = 単リンク法
#@markdown  - `complete`= 完全リンク法
#@markdown  - `ward` = ウォード法
#@markdown
#@markdown `threshold` でクラスタ分けのための距離のしきい値を指定

method = 'single' #@param ['single', 'complete', 'ward']
threshold = 0.7 #@param [0.7, 1.0, 5.0, 7.0, 10.0] {type: "raw"}

# ユークリッド距離を使い，クラスタ間距離の計算に method で指定した方法を用いてクラスタリング
link = hierarchy.linkage(X2, method=method, metric='euclidean')

# threshold の値を距離のしきい値として，学習データをクラスタ分け
label = hierarchy.fcluster(link, t=threshold, criterion='distance')
ncluster = np.max(label)

# グラフ描画の準備
fig = plt.figure(facecolor="white", figsize=(13, 6))

# デンドログラムを描く
ax0 = fig.add_subplot(121)
hierarchy.dendrogram(link, color_threshold=threshold, distance_sort='descending', show_leaf_counts=True, ax=ax0)
ax0.axhline(threshold, color='red', linestyle='--')

# クラスタ分けした学習データを描く
ax1 = fig.add_subplot(122)
for k in range(ncluster):
    XX = X2[label==k+1, :]
    ax1.scatter(XX[:, 0], XX[:, 1])
ax1.text(1.2, 4, f'#cluter = {ncluster}', size=18)
ax1.set_xlim(-5, 5)
ax1.set_ylim(-5, 5)
ax1.set_aspect('equal')

plt.show()

上記のグラフでは，デンドログラムの枝が塗り分けられていますが，その色は右の図のクラスタの塗り分けの色とは一致していません．注意してください．

#### ★★ やってみよう ★★

1. 条件をいろいろ変えてセルを実行してみよう．
1. 次の条件のときに学習データがいくつのクラスタに分けられたか，紙にメモしておこう
- `method = 'single'` で`threshold = 1.0`
- `method = 'ward'` で`threshold = 10.0`
